In [2]:
import numpy as np
import pandas as pd
import time
from cvxopt import matrix, sparse, solvers
import pickle
from matplotlib import pyplot as plt

In [3]:
df = pd.read_csv('processed data.csv')
demand = df['Electricity:Facility [kW](Hourly)'].values
solar = df['Normalized Solar Energy [kWh]'].values
T = len(demand)

In [4]:
# Parameters
C = 5.
xi = .18
sigma = .5
pi = .075

In [5]:
def inequality_matrix(W=0):
    '''
    This function is meant to form a linear programming (sub)problem
    A: The inequality constraint matrix
    b: The inequality constraint bound
    c: The cost function
    '''
    if W <= 0:
        print('Prediction window too short!')
        A = np.array([])
        return A
    
    elif W >= T:
        W = T
        print('Prediction window longer than time length--consider it as offline!')
        
    else:
        pass
    
    # Define the inequality matrix
    A1 = np.zeros((1*W,3*W))
    A2 = np.zeros((1*W,3*W))
    A3 = np.zeros((1*W,3*W))
    for i in range(W):
        A1[i,i*3] = -1.
        A1[i,i*3+1] = 1.
        A1[i,i*3+2] = -1.
        if i >= 1:
            A2[i,:] = A2[i-1,:]
            A3[i,:] = A3[i-1,:]
        A2[i,i*3+1] = 1.
        A3[i,i*3+1] = -1.
        A2[i,i*3+2] = -1.
        A3[i,i*3+2] = 1.
    A4 = -np.eye(3*W)
    A = np.concatenate((A1, A2), axis=0)
    A = np.concatenate((A, A3), axis=0)
    A = np.concatenate((A, A4), axis=0)    
    return A

In [6]:
def inequality_bound(W=0,t=0,s0=0.):
    '''
    inequality bound is an online variable
    it changes according to time t and starting point s0
    '''
    if W <= 0:
        print('Prediction window too short!')
        b = np.array([])
        return b
    
    elif W >= T:
        print('Prediction window longer than time length--consider it as offline!')
        W = T
        # Define the inequality bound
        b1 = solar - demand
        
    else:
        if t-1>=0:
            b1 = solar_extend[t-1:t+W-1] - demand_extend[t-1:t+W-1]
        else:
            b1 = np.append(solar_extend[t-1:],solar_extend[:t+W-1]) - np.append(demand_extend[t-1:],demand_extend[:t+W-1])
    
    b2 = np.ones(W)*(C-s0)
    b3 = np.ones(W)*(s0)
    b4 = np.zeros(3*W)
    b = np.append(b1,b2)
    b = np.append(b,b3)
    b = np.append(b,b4)    
    return b

In [7]:
def cost(W=0):
    
    if W <= 0:
        print('Prediction window too short!')
        c = np.array([])
        return c
    elif W>=T:
        print('Prediction window longer than time length--consider it as offline!')
        W = T
    else:
        pass
    
    # Define the cost function
    c0 = np.array([xi+sigma,pi-sigma,pi+sigma])
    c = np.tile(c0,W)
        
    return c

In [8]:
# Solve offline problem
s0 = 0.
AOFF = inequality_matrix(W=T)
bOFF = inequality_bound(W=T,s0=s0)
cOFF = cost(W=T)

Prediction window longer than time length--consider it as offline!
Prediction window longer than time length--consider it as offline!
Prediction window longer than time length--consider it as offline!


In [9]:
A = matrix(AOFF)
b = matrix(bOFF)
c = matrix(cOFF)

In [10]:
AA = sparse(A)

In [12]:
start = time.time()
sol=solvers.lp(c,AA,b,solver='mosek')
print('Time elasped: %f' % (time.time()-start))

Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : LO (linear optimization problem)
  Constraints            : 52560           
  Cones                  : 0               
  Scalar variables       : 26280           
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.52            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 34.22   
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : LO (linear optimization 

In [13]:
sol['primal objective']

2982.0023672236193

In [3]:
(-3)%3

0